In [1]:
import pandas as pd
from google.cloud import storage, bigquery
import io

# ---------------------------
# CONFIGURATION - UPDATE THESE
# ---------------------------

# 1️⃣ Path to your Google Cloud service account JSON
service_account_path =  "C:\Users\Dell\Desktop\Python Practice\GC Keys\arsipa-dwh-68bb531f3388.json"

# 2️⃣ Google Cloud Storage bucket name
bucket_name = "your-bucket-name"

# 3️⃣ File path inside the bucket (e.g., "folder/file.csv")
file_name = "folder/your_file.csv"

# 4️⃣ BigQuery details
project_id = "your-gcp-project-id"
dataset_id = "your_dataset"
table_id = "your_table"

# ---------------------------
# Initialize GCS client
# ---------------------------

storage_client = storage.Client.from_service_account_json(service_account_path)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download file into memory
data_bytes = blob.download_as_bytes()

# Read CSV into pandas DataFrame
df = pd.read_csv(io.BytesIO(data_bytes))

# Preview
print("Data preview:")
print(df.head())

# ---------------------------
# Initialize BigQuery client
# ---------------------------

bq_client = bigquery.Client.from_service_account_json(service_account_path)
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Optional: overwrite table if it exists
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

# Load DataFrame to BigQuery
job = bq_client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for job to finish

print(f"Loaded {job.output_rows} rows into {table_ref}.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/path/to/your/service_account.json'

In [3]:
import pandas as pd
import gcsfs

fs = gcsfs.GCSFileSystem(project="test-project")  # just for testing
with fs.open("local_test_folder/test.csv", "w") as f:
    f.write("col1,col2\n1,2\n3,4")

df = pd.read_csv("local_test_folder/test.csv")
print(df)

FileNotFoundError: https://storage.googleapis.com/upload/storage/v1/b/local_test_folder/o?name=test.csv

In [5]:
from minio import Minio
import pandas as pd
import io

# Connect to MinIO server
client = Minio(
    "127.0.0.1:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False  # because running locally
)

# Create bucket (if not exists)
bucket_name = "test-bucket"
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)

# Upload a CSV file
csv_data = "id,name\n1,John\n2,Jane\n"
client.put_object(
    bucket_name,
    "sample.csv",
    io.BytesIO(csv_data.encode("utf-8")),
    length=len(csv_data),
    content_type="text/csv"
)

# Download and read CSV into pandas
response = client.get_object(bucket_name, "sample.csv")
df = pd.read_csv(io.BytesIO(response.read()))
response.close()
response.release_conn()

print(df)


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=9000): Max retries exceeded with url: /test-bucket?location= (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021AAFA88CD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
from google.cloud import storage, bigquery
import pandas as pd
import io

# ---------------------------
# Configuration
# ---------------------------

# Path to your service account key JSON file
service_account_path = "C:\Users\Dell\Desktop\Python Practice\GC Keys\arsipa-dwh-68bb531f3388.json"

# GCS bucket and file
bucket_name = "your-bucket-name"
file_name = "folder/your_file.csv"  # Can be .csv, .xlsx, etc.

# BigQuery details
project_id = "your-gcp-project-id"
dataset_id = "your_dataset"
table_id = "your_table"

# ---------------------------
# Initialize clients
# ---------------------------

# GCS client
storage_client = storage.Client.from_service_account_json(service_account_path)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download file into memory
data_bytes = blob.download_as_bytes()

# Assuming CSV file, read into pandas
df = pd.read_csv(io.BytesIO(data_bytes))

# Preview data
print(df.head())

# ---------------------------
# BigQuery client
# ---------------------------

bq_client = bigquery.Client.from_service_account_json(service_account_path)

# Define table reference
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Upload DataFrame to BigQuery
job = bq_client.load_table_from_dataframe(df, table_ref)
job.result()  # Wait for the job to complete

print(f"Loaded {job.output_rows} rows into {table_ref}.")
